In [ ]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import dateparser
import unidecode
import time
import random

# Chrome driver configuration
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

def find_element_text_or_nan(element, selector):
    """ Try to find and return the text of an element, or NaN if not found. """
    try:
        return element.find_element(By.CSS_SELECTOR, selector).text
    except NoSuchElementException:
        return np.nan

data_folder = "data"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Loop through pages (insert numbers)
for page_num in range(4, 12347):
    url = f"https://www.vie-publique.fr/discours?page={page_num}"
    driver.get(url)

    # Save HTML content of the list page
    html_folder = os.path.join(data_folder, "html_page")
    if not os.path.exists(html_folder):
        os.makedirs(html_folder)
    html_filename = os.path.join(html_folder, f"html_page_{page_num}.html")
    with open(html_filename, "w", encoding="utf-8") as file:
        file.write(driver.page_source)

    # Find all elements with the class 'fr-card__content'
    cards = driver.find_elements(By.CLASS_NAME, "fr-card__content")

    # List to store initial data
    data = []

    for card in cards:
        # Using the function to find text or return NaN
        nature = find_element_text_or_nan(card, ".fr-card__start .field__item")
        title = find_element_text_or_nan(card, ".fr-card__title a")
        url = find_element_text_or_nan(card, ".fr-card__title a")

        # Parsing date separately as it involves additional logic
        date_text = find_element_text_or_nan(card, ".fr-card__end time")
        date = dateparser.parse(date_text).strftime("%d/%m/%Y") if dateparser.parse(date_text) else np.nan

        data.append({"nature": nature, "title": title, "url": url, "date": date})

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Loop through each link to extract additional information
    for index, row in df.iterrows():
        driver.get(row["url"])

        # Sleep for a random duration between 1 and 5 seconds (let's hope I don't get banned=>toolongdontcare)
        #time.sleep(random.uniform(1, 2))

        # Save HTML content of the current link page
        link_html_folder = os.path.join(data_folder, "url_html_page")
        if not os.path.exists(link_html_folder):
            os.makedirs(link_html_folder)
        link_html_filename = os.path.join(link_html_folder, f"html_page_{page_num}_url_{index}.html")
        with open(link_html_filename, "w", encoding="utf-8") as file:
            file.write(driver.page_source)

        # Extract 'tag'
        try:
            tag = driver.find_element(By.CLASS_NAME, "vp-item-tag").text
        except NoSuchElementException:
            tag = "NA"
        df.at[index, 'tag'] = tag

        # Extract 'speaker'
        try:
            speakers_elements = driver.find_elements(By.CSS_SELECTOR, ".line-intervenant li")
            speakers = []
            for speaker_element in speakers_elements:
                speaker_name = speaker_element.find_element(By.CSS_SELECTOR, "a").text.strip()
                speaker_title = speaker_element.text.split(" - ")[-1].split(";")[0].strip()
                speakers.append(f"{speaker_name} - {speaker_title}")
            df.at[index, 'speaker'] = " ; ".join(speakers)
        except NoSuchElementException:
            df.at[index, 'speaker'] = "NA"

        # Check if 'vp-intervenant' exists and get its position
        try:
            intervenant_element = driver.find_element(By.CLASS_NAME, "vp-intervenant")
            intervenant_position = intervenant_element.location['y']
        except NoSuchElementException:
            intervenant_position = None

        # Find the last <p> tag after 'vp-intervenant'
        try:
            all_p_elements = driver.find_elements(By.CSS_SELECTOR, ".vp-discours-details p")
            for p_element in reversed(all_p_elements):
                if intervenant_position and p_element.location['y'] > intervenant_position:
                    last_p_text = p_element.text
                    break
            else:
                raise NoSuchElementException  # No <p> found after 'vp-intervenant'

            column_name = unidecode.unidecode(last_p_text.split(":")[0].strip().split()[0].lower()) # Remove accents + lower case
            column_value = last_p_text.split(":")[1].strip() if ":" in last_p_text else "NA"
            df.at[index, column_name] = column_value
        except (NoSuchElementException, IndexError):
            pass  # Do not create column if the element is absent or there is an error

        # Extract 'text'
        try:
            text = driver.find_element(By.CLASS_NAME, "field--name-field-texte-integral").text
        except NoSuchElementException:
            text = "NA"
        df.at[index, 'text'] = text

        # Extract 'keywords'
        try:
            keywords_elements = driver.find_elements(By.CLASS_NAME, "fr-tag--green-emeraude")
            keywords = " ; ".join([keyword.text for keyword in keywords_elements])
        except NoSuchElementException:
            keywords = "NA"
        df.at[index, 'keywords'] = keywords

    # Save the DataFrame in CSV in the 'data/csv_page' subfolder
    csv_folder = os.path.join(data_folder, "csv_page")
    if not os.path.exists(csv_folder):
        os.makedirs(csv_folder)
    csv_filename = os.path.join(csv_folder, f"csv_page_{page_num}.csv")
    df.to_csv(csv_filename, index=False)

# Close the browser
driver.quit()


In [86]:
df

,nature,title,url,date,tag,speaker,circonstance,text,keywords,media
0,Déclaration,Déclaration à la presse de Mme Catherine Colon...,https://www.vie-publique.fr/discours/292401-ca...,11/12/2023,International,Catherine Colonna - Ministre de l'Europe et de...,Arrivée au Conseil Affaires étrangères,Je ne vous apprendrai rien si je vous dis que ...,International ; Union européenne ; Constructio...,NaN
1,Déclaration,"Déclaration de Mme Catherine Colonna, ministre...",https://www.vie-publique.fr/discours/292400-ca...,10/12/2023,Institutions,Catherine Colonna - Ministre de l'Europe et de...,75ème anniversaire de la Déclaration universel...,"Merci, Madame l'Ambassadrice, merci, chère De...",Institutions ; Justice - Droits fondamentaux ;...,NaN
2,Déclaration,"Déclaration de M. Emmanuel Macron, président d...",https://www.vie-publique.fr/discours/292372-em...,10/12/2023,Institutions,Emmanuel Macron - Président de la République,75e anniversaire de la Déclaration des Droits ...,Mesdames et Messieurs les représentants d'orga...,Institutions ; Justice - Droits fondamentaux ;...,NaN
3,Interview,"Entretien de M. Olivier Becht, ministre délégu...",https://www.vie-publique.fr/discours/292394-en...,08/12/2023,International,Olivier Becht - Ministre délégué auprès de la ...,NaN,Q - Comment améliorer l'attractivité de la Fra...,International ; Economie internationale ; Comm...,BFM Business
4,Communiqué,"Communiqué de la Présidence de la République, ...",https://www.vie-publique.fr/discours/292297-pr...,08/12/2023,International,Présidence de la République - Présidence de la...,Entretien téléphonique avec Benyamin Netanyaho...,"Le Président de la République s'est entretenu,...",International ; Asie ; Israël ; France - Israë...,NaN
5,Déclaration,"Déclaration de M. Bruno Le Maire, ministre de ...",https://www.vie-publique.fr/discours/292365-br...,08/12/2023,International,"Bruno Le Maire - Ministre de l'économie, des f...",Arrivée au Conseil des ministres de l'Union eu...,"M. Bruno Le Maire : ""Bonjour à toutes et à tou...",International ; Union européenne ; Constructio...,NaN
6,Communiqué,"Communiqué de la Présidence de la République, ...",https://www.vie-publique.fr/discours/292366-pr...,08/12/2023,Société,Présidence de la République - Présidence de la...,Déplacement du Président Emmanuel Macron sur l...,"Le vendredi 8 décembre, le Président de la Rép...",Société ; Culture - Médias ; Protection du pat...,NaN
7,Déclaration,"Déclaration de Mme Élisabeth Borne, Première m...",https://www.vie-publique.fr/discours/292290-el...,08/12/2023,Société,Élisabeth Borne - Première ministre,Visite à Mayotte le 8 décembre 2023,"Bonjour à tous.\nVous le savez, depuis plusieu...",Société ; Environnement ; Eau ; Mayotte ; Séch...,NaN
8,Interview,"Interview de Mme Agnès Pannier-Runacher, minis...",https://www.vie-publique.fr/discours/292367-ag...,08/12/2023,Economie,Agnès Pannier-Runacher - Ministre de la transi...,NaN,ROMAIN DESARBRES\nBonjour Agnès PANNIER-RUNACH...,Economie ; Energie - Transports ; Politique de...,Europe 1
9,Interview,Extrait d'une interview de M. Jean-Noël Barrot...,https://www.vie-publique.fr/discours/292364-je...,08/12/2023,Société,"Jean-Noël Barrot - Ministre délégué, chargé du...",NaN,L'Étudiant : Pourquoi le numérique peine-t-il ...,Société ; Sciences - Numérique - I. A. ; Econo...,L'Etudiant
